<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Gen AI Experiments](https://img.shields.io/badge/Gen%20AI%20Experiments-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://github.com/buildfastwithai/gen-ai-experiments)
[![Gen AI Experiments GitHub](https://img.shields.io/github/stars/buildfastwithai/gen-ai-experiments?style=for-the-badge&logo=github&color=gold)](http://github.com/buildfastwithai/gen-ai-experiments)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/[NOTEBOOK_ID])

## Master Generative AI in 8 Weeks
**What You'll Learn:**
- Master cutting-edge AI tools & frameworks
- 6 weeks of hands-on, project-based learning
- Weekly live mentorship sessions
- No coding experience required
- Join Innovation Community

Transform your AI ideas into reality through hands-on projects and expert mentorship.

[Start Your Journey](https://www.buildfastwithai.com/genai-course)

---

# Gemini 3 Pro - Testing & Basics

**Created by:** @BuildFastWithAI  
**Model:** Google Gemini 3 Pro  
**Last Updated:** November 2025

This notebook covers the fundamentals of working with Gemini 3 Pro, including:
- Basic setup and hello world examples
- Tool calling capabilities
- Simple agent implementation
- Quick RAG demo
- Performance metrics and cost estimates

## 1. Setup & Installation

In [ ]:
!pip install -q google-generativeai langchain langchain-google-genai faiss-cpu

In [ ]:
import google.generativeai as genai
from google.colab import userdata
import time
import json

# Configure API key from Colab secrets
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("✅ API key configured successfully")
except Exception as e:
    print(f"❌ Error: {e}")
    print("Please add GOOGLE_API_KEY to Colab secrets")

## 2. Basic Example - Hello World

In [ ]:
# Initialize the model
model = genai.GenerativeModel('gemini-3-pro')

# Simple generation
response = model.generate_content("Hello! Introduce yourself and explain what makes you unique.")
print(response.text)

In [ ]:
# Test with parameters
generation_config = {
    "temperature": 0.7,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 1024,
}

model_configured = genai.GenerativeModel(
    'gemini-3-pro',
    generation_config=generation_config
)

response = model_configured.generate_content(
    "Write a creative short story about AI in 3 sentences."
)
print(response.text)

## 3. Tool Calling

In [ ]:
# Define tools
def get_weather(location: str) -> dict:
    """Get weather information for a location."""
    # Simulated response
    return {
        "location": location,
        "temperature": 72,
        "condition": "Sunny",
        "humidity": 45
    }

def calculator(operation: str, num1: float, num2: float) -> float:
    """Perform basic arithmetic operations."""
    operations = {
        "add": num1 + num2,
        "subtract": num1 - num2,
        "multiply": num1 * num2,
        "divide": num1 / num2 if num2 != 0 else "Error: Division by zero"
    }
    return operations.get(operation, "Invalid operation")

# Define function declarations for Gemini
tools = [
    {
        "function_declarations": [
            {
                "name": "get_weather",
                "description": "Get weather information for a specific location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "City name or location"
                        }
                    },
                    "required": ["location"]
                }
            },
            {
                "name": "calculator",
                "description": "Perform arithmetic operations",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "operation": {
                            "type": "string",
                            "enum": ["add", "subtract", "multiply", "divide"]
                        },
                        "num1": {"type": "number"},
                        "num2": {"type": "number"}
                    },
                    "required": ["operation", "num1", "num2"]
                }
            }
        ]
    }
]

model_with_tools = genai.GenerativeModel('gemini-3-pro', tools=tools)
chat = model_with_tools.start_chat()

response = chat.send_message("What's the weather in San Francisco and calculate 25 * 4?")
print(response.text)

## 4. Simple Agent Implementation

In [ ]:
# Simple ReAct agent
class SimpleAgent:
    def __init__(self, model_name='gemini-3-pro'):
        self.model = genai.GenerativeModel(model_name, tools=tools)
        self.chat = self.model.start_chat()
        self.tool_map = {
            "get_weather": get_weather,
            "calculator": calculator
        }
    
    def run(self, query: str, max_iterations: int = 5):
        print(f"Query: {query}\n")
        
        for i in range(max_iterations):
            response = self.chat.send_message(query)
            
            # Check if model wants to call a function
            if hasattr(response, 'parts'):
                for part in response.parts:
                    if hasattr(part, 'function_call'):
                        func_call = part.function_call
                        func_name = func_call.name
                        func_args = dict(func_call.args)
                        
                        print(f"🔧 Calling tool: {func_name}")
                        print(f"   Args: {func_args}")
                        
                        # Execute function
                        result = self.tool_map[func_name](**func_args)
                        print(f"   Result: {result}\n")
                        
                        # Send result back to model
                        query = str(result)
                        continue
            
            # If no function call, we have final answer
            print(f"Final Answer: {response.text}")
            break

# Test agent
agent = SimpleAgent()
agent.run("What's the weather in New York and what's 150 divided by 5?")

## 5. Quick RAG Demo

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA

# Sample documents
documents = [
    "Gemini 3 Pro is Google's latest large language model with enhanced reasoning capabilities.",
    "The model supports multimodal inputs including text, images, and audio.",
    "Gemini 3 Pro offers improved context understanding and can handle up to 1M tokens.",
    "It features advanced function calling and structured output generation.",
    "The model is optimized for low latency and cost-effective inference."
]

# Create embeddings and vector store
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
texts = text_splitter.create_documents(documents)

vectorstore = FAISS.from_documents(texts, embeddings)

# Create QA chain
llm = ChatGoogleGenerativeAI(model="gemini-3-pro", temperature=0.3)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3})
)

# Query
query = "What are the key features of Gemini 3 Pro?"
response = qa_chain.run(query)
print(f"Question: {query}")
print(f"\nAnswer: {response}")

## 6. Use Cases

### Customer Support Agent

In [ ]:
customer_support_prompt = """
You are a helpful customer support agent. Assist users with their queries professionally.
User Query: {query}
"""

support_model = genai.GenerativeModel('gemini-3-pro')
query = "I can't log into my account. What should I do?"
response = support_model.generate_content(customer_support_prompt.format(query=query))
print(response.text)

### Code Assistant

In [ ]:
code_prompt = "Write a Python function to find the nth Fibonacci number with memoization."
response = model.generate_content(code_prompt)
print(response.text)

### Data Analysis

In [ ]:
data = {
    "sales": [1000, 1500, 1200, 1800, 2000],
    "months": ["Jan", "Feb", "Mar", "Apr", "May"]
}

analysis_prompt = f"""
Analyze this sales data and provide insights:
{json.dumps(data, indent=2)}

Include:
1. Trends
2. Growth rate
3. Recommendations
"""

response = model.generate_content(analysis_prompt)
print(response.text)

## 7. Performance Metrics & Cost Analysis

In [ ]:
import time

# Benchmark
test_prompts = [
    "What is machine learning?",
    "Explain quantum computing in simple terms.",
    "Write a haiku about AI."
]

total_time = 0
total_tokens = 0

for prompt in test_prompts:
    start = time.time()
    response = model.generate_content(prompt)
    elapsed = time.time() - start
    total_time += elapsed
    
    # Estimate tokens (rough calculation)
    tokens = len(prompt.split()) + len(response.text.split())
    total_tokens += tokens
    
    print(f"Prompt: {prompt[:50]}...")
    print(f"Time: {elapsed:.2f}s | Est. Tokens: {tokens}\n")

print(f"\n📊 Summary:")
print(f"Total Time: {total_time:.2f}s")
print(f"Average Time: {total_time/len(test_prompts):.2f}s")
print(f"Total Tokens: ~{total_tokens}")
print(f"\n💰 Estimated Cost (assuming $0.001/1K tokens): ${(total_tokens/1000) * 0.001:.4f}")

## 8. Key Takeaways

✅ **Strengths:**
- Fast response times and low latency
- Excellent tool calling capabilities
- Strong reasoning and context understanding
- Cost-effective for production use

📌 **Best Practices:**
- Use Colab secrets for API keys
- Implement error handling for production
- Monitor token usage and costs
- Test with different temperature settings

🔗 **Resources:**
- [Official Documentation](https://ai.google.dev/)
- [API Reference](https://ai.google.dev/api)
- Follow [@BuildFastWithAI](https://twitter.com/BuildFastWithAI) for more tutorials